In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from scipy.optimize import minimize
from scipy.optimize import least_squares


In [3]:
from numpy.random import default_rng, SeedSequence

sq = SeedSequence()
seed = sq.entropy        # on sauve la graine pour reproduire les résultats
print('seed = ', seed)
rng = default_rng(sq)
rng.standard_normal(5)

seed =  226259853752425655923013014822859208066


array([-0.48982392, -1.31010709, -0.22355921, -0.10418999, -0.27984759])

In [4]:
sns.set_style("whitegrid")
mpl.rcParams['figure.dpi'] = 100

params_grid = {"color": 'lightgrey', "linestyle": 'dotted', "linewidth": 0.7 }

In [5]:
# Pamètre du model
N = 10      
M = 10000
T = 1
S0 = 100
K = 100
sigma = 0.1
r = 0.04

We are going to code the Longstaff-Schwartz algorithm for an American Put with respect to a Black Scholes model, that is to say a put that we can exerce at $\frac{k}/N$ where $k = 0,..., N$. We will take $r=0.04$, $\sigma=0.1$, $x_0=100$ and the strike $K=100$ with $N=10$ dates until $T=1$ and the payoff $\phi_k(x) = e^{-r k/N}(K-x)_+$.

In [6]:
def brownian_1d(n_times: int, n_paths: int, 
                final_time: float=1.0, 
                increments: bool=False, 
                random_state: np.random.Generator=rng) -> np.array:
    """Simulate paths of standard Brownian motion
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths 
        final_time: Final time of simulation
        increments: If `True` the increments of the paths are returned.
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths if the argument `increments` is `False`
        `np.array` of shape `(n_times, n_paths)` containing the increments if the argument `increments` is `True`
    """
    dB = np.sqrt(final_time / n_times) * random_state.standard_normal((n_times, n_paths))
    if increments:
        return dB
    else:
        brownian = np.zeros((n_times+1, n_paths))
        brownian[1:] = np.cumsum(dB, axis=0)
        return brownian

def black_scholes_1d(n_times: int, n_paths: int, 
                     final_time: float=1.0, 
                     random_state: np.random.Generator=rng, *,
                     init_value: float,
                     r: float, sigma: float) -> np.array:
    """Simulate paths of Black-Scholes process
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths 
        final_time: Final time of simulation
        init_value: `S0`
        r: Interest rate
        sigma: Volatility
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths 
    """
    Bt = brownian_1d(n_times, n_paths)
    times = np.arange(n_times+1)*(1/n_times)
    t = times[:, np.newaxis]
    St = init_value * np.exp((r - 0.5*sigma**2)*t + sigma*Bt)
    return St

To find the value $V_0$, we apply the following backward recurrence : 
$V_N(x) = \phi_N(x)$

and $V_n(x) = max(\phi_n(x), \mathbb{E}[V_{n+1}(x))$

The aim of the LS algorithm is to approch the conditional expectation with function.

For that we use the following property : 
$$\theta^* = \argmin_{\theta} \mathbb{E}[(\mathbb{E}(Y|X) - \Phi(X, \theta))^2]$$
$$ = \argmin_{\theta} \mathbb{E}[(Y - \Phi(X, \theta))^2]$$
In the following example we will use Linear Regression, the algorithm become :
$$V_N(x) = \phi_N(x)$$

$$V_n(x) = max(\phi_n(x), \Phi(x,\theta_n))$$

with $\theta_n = argmin_\theta \sum_{j=1}^{M} \left( V_{n+1}(X_{n+1}^{(j)}) - \Phi(X_{n}^{(j)}, \theta) \right)^2$

For $\Phi$ we can choose between two families of function : 

$\Phi(x) = \theta_0+\theta_1*x+\theta_2*x^2$ 

or

 $\Phi(x) = \theta_0+\theta_1*x+\theta_2*x^2+\theta_3*max(K-x, 0)$

In the following, I used the second version od $\Phi$ but we will found the result for the first version.

In [7]:
def psi(x, theta, K):
    return theta[0]+theta[1]*x+theta[2]*(x**2)+theta[3]*max(K-x, 0)

def phi(x, K, k, N):
    return np.exp(-r*k/N)*np.maximum(K-x, 0)

#same function as the previous one but it allows to manipulate matrix
#it will be useful when I will use a regression to find theta based on the prices obtained by Black Scholes simulation
def psi_regression(x, K, k, N): 
    return np.exp(-r*k/N)*np.maximum(K-x[k], 0)

#Function where we want to find the argmin theta
def objective_function(theta, M, S, n, K, thetas):
    result = 0
    for j in range(M):
        result += (V(S[n+1][j], n+1, thetas[-1], N, K) - psi(S[n][j], theta, K))**2
    return result/M

def V(x, n, theta,N, K):
    if n == N:
        return phi(x,K,N,N)
    else:
        return max(phi(x, K,n,N), psi(x, theta, K))
    

# Avec minimize de Scipy

In [8]:
def longstaff_Schwartz_scipy(x0, K, T, N, M):
    V_hat = [phi(x0, K,N, N)]
    S_t = black_scholes_1d(n_times=N, n_paths=M, final_time= T, init_value=x0, r = r, sigma = sigma)
    thetas = [[0,0,0,0]]# For n  =N, V_N does not depend on theta
    for n in reversed(range(N)):
        print(n)
        initial_theta = np.ones(4)

        theta = minimize(objective_function, initial_theta, args=(M, S_t, n, K, thetas))
        optimized_theta = theta.x
        print(optimized_theta)
        V_hat.append(max(phi(x0, K,n,N), psi(x0, optimized_theta, K)))
        thetas.append(optimized_theta)
    return V_hat, thetas

In [9]:
price_scipy = longstaff_Schwartz_scipy(S0, K, T, N, M)
print(price_scipy)

9
[ 2.37372402e+01 -4.05713714e-01  1.72676791e-03  8.16797292e-01]
8
[ 4.32122287e+01 -7.45231036e-01  3.20625312e-03  7.16885567e-01]
7
[ 7.06804417e+01 -1.23341041e+00  5.37464996e-03  6.18941091e-01]
6
[ 9.59281106e+01 -1.68583303e+00  7.40312063e-03  5.23851613e-01]
5
[ 1.21803840e+02 -2.15241070e+00  9.50843427e-03  4.28960596e-01]
4
[ 1.43438577e+02 -2.54847156e+00  1.13265062e-02  3.66134059e-01]
3
[ 1.66808675e+02 -2.97487531e+00  1.32782207e-02  2.81154647e-01]
2
[ 1.77647961e+02 -3.16011363e+00  1.40716877e-02  2.04393448e-01]
1
[ 0.97815136  0.27182903 -0.00257719  0.37235418]
0
[ 1.00170248  1.00193807 -0.00985062  1.        ]
([0.0, 0.43354787726703137, 0.7516563564688425, 1.0859003203095412, 1.3760137577170894, 1.6471120703406683, 1.8564824249237262, 2.103350603907586, 2.3534746703528526, 2.3891759993670547, 2.6892986731221953], [[0, 0, 0, 0], array([ 2.37372402e+01, -4.05713714e-01,  1.72676791e-03,  8.16797292e-01]), array([ 4.32122287e+01, -7.45231036e-01,  3.20625312

In [10]:
print("Le prix d'un Put bermudéen avec scipy est ", price_scipy[0][-1], "$")

Le prix d'un Put bermudéen avec scipy est  2.6892986731221953 $


Le Prix est un peu long pour être obtenu mais est proche du prix théorique obtenu par le modèle CRR (on obtient 2.61$ avec ce modèle)(cf TP1)

# Avec Regression Linéaire

Pour pouvoir effectuer une régression linéaire il faut générer des valeurs d'entrainements pour V. Pour cela on utilise la récurrence backward suivante :

$V_N(x) = \phi_N(x)$

$V_n(x) = max(\phi_n(x), \mathbb{E}\left[V_{n+1} \mid \mathcal{F}_n\right] )$ 

In [330]:
def simulate_V(S,K,r,n,V):
    if n==-1:
        return V
    else : 
        V[n] = np.maximum(psi_regression(S,K,n,N),V[n+1])
        return simulate_V(S,K,r,n-1,V)

def theta_regressions(X, y, K):
    n_times = X.shape[0]
    theta_matrix = np.zeros((n_times-1, 4))

    # EStimateur OlS
    theta_matrix[1:, :] = [
        np.linalg.inv(np.c_[np.ones((X_i.shape[0], 1)), X_i, X_i**2, np.maximum(K - X_i, 0)].T.dot(np.c_[np.ones((X_i.shape[0], 1)), X_i, X_i**2, np.maximum(K - X_i, 0)]))
        .dot(np.c_[np.ones((X_i.shape[0], 1)), X_i, X_i**2, np.maximum(K - X_i, 0)].T)
        .dot(y[i, :].reshape(-1, 1)).flatten()
        for i, X_i in enumerate(X[1:-1, :])
    ]

    return theta_matrix

def longstaff_Schwartz_regression(x0, K, T, N, M):
    S_t = black_scholes_1d(n_times=N, n_paths=M, final_time= T, init_value=x0, r = r, sigma = sigma)
    thetas = []

    V_simu = np.zeros_like(S_t)
    V_simu[-1] = psi_regression(S_t, K, N, N)

    V_simu = simulate_V(S_t,K,r,V_simu.shape[0]-2,V_simu)
    theta_matrix = theta_regressions(S_t, V_simu, K)
    print(theta_matrix)
    V_hat = []
    for n in range(N):
        print(n)
        
        optimized_theta = theta_matrix[n]
        print(optimized_theta)
        V_hat.append(max(phi(x0, K,n,N), psi(x0, optimized_theta, K)))
        thetas.append(optimized_theta)
    return V_hat, thetas

In [331]:
price_Gradient = longstaff_Schwartz_regression(S0, K, T, N, M)
print(price_Gradient)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.40248629e+02 -6.00904997e+00  2.65069579e-02  3.40462450e-02]
 [ 2.86002987e+02 -5.00272677e+00  2.18623665e-02  8.72861214e-02]
 [ 2.80607798e+02 -4.91355080e+00  2.14894321e-02  9.24497506e-02]
 [ 2.25877128e+02 -3.90786059e+00  1.68710055e-02  2.17980547e-01]
 [ 1.84366639e+02 -3.15996978e+00  1.35027677e-02  3.31730065e-01]
 [ 1.50491582e+02 -2.56173482e+00  1.08630851e-02  4.46915396e-01]
 [ 1.18539593e+02 -2.00500123e+00  8.44263229e-03  5.37298032e-01]
 [ 1.00056028e+02 -1.69000232e+00  7.10316360e-03  5.95624978e-01]
 [ 6.52775213e+01 -1.09399363e+00  4.55913577e-03  7.16158477e-01]]
0
[0. 0. 0. 0.]
1
[ 3.40248629e+02 -6.00904997e+00  2.65069579e-02  3.40462450e-02]
2
[ 2.86002987e+02 -5.00272677e+00  2.18623665e-02  8.72861214e-02]
3
[ 2.80607798e+02 -4.91355080e+00  2.14894321e-02  9.24497506e-02]
4
[ 2.25877128e+02 -3.90786059e+00  1.68710055e-02  2.17980547e-01]
5
[ 1.84366639e+02 -3.15996978e+00  1.350

In [332]:
print("Le prix d'un Put bermudéen avec régression est ", price_Gradient[0][-1], "$")

Le prix d'un Put bermudéen avec régression est  1.4695162139262692 $


On trouve une valeur bien plus faible que précédements, j'ai donc du faire une erreur dans mon inplémentation nottament lors de la régression car mes coefficients theta sont éloignés de ceux trouver dans la partie précédente

# Même chose avec la première version de $\Phi$

## D'abord avec Scipy 

In [317]:
def psi_V2(x, theta, K):
    return theta[0]+theta[1]*x+theta[2]*(x**2)

# Algorithme de Longstaff-Scwartz
def objective_function(theta, M, S, n, K, thetas):
    result = 0
    for j in range(M):
        result += (V(S[n+1][j], n+1, thetas[-1], N, K) - psi_V2(S[n][j], theta, K))**2
    return result/M

def V(x, n, theta,N, K):
    if n == N:
        return phi(x,K,N,N)
    else:
        return max(phi(x, K,n,N), psi_V2(x, theta, K))

def longstaff_Schwartz_V2(x0, K, T, N, M):
    V_hat = [phi(x0, K,N, N)]
    S_t = black_scholes_1d(n_times=N, n_paths=M, final_time= T, init_value=x0, r = r, sigma = sigma)
    thetas = [[0,0,0]] #Peut importe les valeurs, on V_N ne dépend pas de theta
    for n in reversed(range(N)):
        print(n)
        initial_theta = np.ones(3)

        theta = minimize(objective_function, initial_theta, args=(M, S_t, n, K, thetas))
        optimized_theta = theta.x
        print(optimized_theta)
        V_hat.append(max(phi(x0, K,n,N), psi_V2(x0, optimized_theta, K)))
        thetas.append(optimized_theta)
    return V_hat, thetas

In [318]:
price_V2 = longstaff_Schwartz_V2(S0, K, T, N, M)

9
[ 2.06082540e+02 -3.61992695e+00  1.58077619e-02]
8
[ 2.23204013e+02 -3.95171459e+00  1.74499518e-02]
7
[ 2.38385680e+02 -4.24831526e+00  1.89098965e-02]
6
[ 2.48197194e+02 -4.44393834e+00  1.98936317e-02]
5
[ 2.55726962e+02 -4.60276634e+00  2.07326016e-02]
4
[ 2.62194890e+02 -4.74111549e+00  2.14749719e-02]
3
[ 2.67969094e+02 -4.86554830e+00  2.21499233e-02]
2
[ 2.61135909e+02 -4.74814158e+00  2.16659472e-02]
1
[ 2.42339370e+02 -4.38995182e+00  1.99709879e-02]
0
[ 0.99770046  0.99385543 -0.00973002]


In [319]:
print("Le prix d'un Put bermudéen avec scipy est ", price_V2[0][-1], "$")

Le prix d'un Put bermudéen avec scipy est  3.0830534726197243 $


## Ensuite avec Régression OLS

In [339]:
def simulate_V_V2(S,K,r,n,V):
    if n==-1:
        return V
    else : 
        V[n] = np.maximum(psi_regression(S,K,n,N),V[n+1])
        return simulate_V(S,K,r,n-1,V)

def theta_regressions_V2(X, y, K):
    n_times = X.shape[0]
    theta_matrix = np.zeros((n_times-1, 3))

    # EStimateur OlS
    theta_matrix[1:, :] = [
        np.linalg.inv(np.c_[np.ones((X_i.shape[0], 1)), X_i, X_i**2].T.dot(np.c_[np.ones((X_i.shape[0], 1)), X_i, X_i**2]))
        .dot(np.c_[np.ones((X_i.shape[0], 1)), X_i, X_i**2].T)
        .dot(y[i, :].reshape(-1, 1)).flatten()
        for i, X_i in enumerate(X[1:-1, :])
    ]

    return theta_matrix

def longstaff_Schwartz_regression_V2(x0, K, T, N, M):
    S_t = black_scholes_1d(n_times=N, n_paths=M, final_time= T, init_value=x0, r = r, sigma = sigma)
    thetas = []

    V_simu = np.zeros_like(S_t)
    V_simu[-1] = psi_regression(S_t, K, N, N)

    V_simu = simulate_V(S_t,K,r,V_simu.shape[0]-2,V_simu)
    theta_matrix = theta_regressions_V2(S_t, V_simu, K)
    print(theta_matrix)
    V_hat = []
    for n in range(N):
        print(n)
        
        optimized_theta = theta_matrix[n]
        print(optimized_theta)
        V_hat.append(max(phi(x0, K,n,N), psi_V2(x0, optimized_theta, K)))
        thetas.append(optimized_theta)
    return V_hat, thetas

In [340]:
price_V2_reg = longstaff_Schwartz_regression_V2(S0, K, T, N, M)
print(price_V2_reg)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.81379665e+02 -6.82513988e+00  3.05552024e-02]
 [ 3.35988075e+02 -5.95694569e+00  2.64106920e-02]
 [ 3.20125843e+02 -5.65920404e+00  2.50061283e-02]
 [ 3.02459481e+02 -5.32432706e+00  2.34084441e-02]
 [ 2.90037760e+02 -5.10076255e+00  2.23878242e-02]
 [ 2.73295700e+02 -4.78660001e+00  2.09065506e-02]
 [ 2.61492278e+02 -4.57002051e+00  1.99002331e-02]
 [ 2.48200169e+02 -4.33010628e+00  1.88044717e-02]
 [ 2.28631077e+02 -3.97230532e+00  1.71638292e-02]]
0
[0. 0. 0.]
1
[ 3.81379665e+02 -6.82513988e+00  3.05552024e-02]
2
[ 3.35988075e+02 -5.95694569e+00  2.64106920e-02]
3
[ 3.20125843e+02 -5.65920404e+00  2.50061283e-02]
4
[ 3.02459481e+02 -5.32432706e+00  2.34084441e-02]
5
[ 2.90037760e+02 -5.10076255e+00  2.23878242e-02]
6
[ 2.73295700e+02 -4.78660001e+00  2.09065506e-02]
7
[ 2.61492278e+02 -4.57002051e+00  1.99002331e-02]
8
[ 2.48200169e+02 -4.33010628e+00  1.88044717e-02]
9
[ 2.28631077e+02 -3.97230532e+00  1.71638292e-02]
([0.0, 4

In [341]:
print("Le prix d'un Put bermudéen avec régression est ", price_V2_reg[0][-1] , "$")

Le prix d'un Put bermudéen avec régression est  3.0388372710069973 $
